<img src="./img/CCIT_Logo.png" alt="CCit Logo" width=400>

# 04 - Differential Power Analysis (DPA) 🕵️

## ⚠️ Prerequisites

Hol' up - before continuing, ensure you have done the following:

* ☑ Read the introduction to Side-Channel Analysis in the [02-Intro to SCA](./02-Intro%20to%20SCA.ipynb) notebook.
* ☑ Learn how Simple Power Analysis works in [03-Simple Power Analysis](./03-Simple%20Power%20Analysis.ipynb) notebook.

### 📑 Summary

*In this lab, you'll learn what is Differential Power Analysis, AKA* **DPA**, *how to carry out such an attack and why it is the foundation of all attacks used to retrieve secret keys from cipher implementations. Using our remote ChipWhisperer board, we will capture enough traces to retrieve the secret key from the `AES-128` algorithm running in the target microcontroller. We will build a complete Differential attack from scratch, to deeply understand what is the rationale behind it.*

### 💬 Learning outcomes
* What is DPA
* How to carry out such an attack
* What is the rationale behind, what phenomena it exploits
* How to write, from scratch, a complete Differential Power Attack

## 📑 Table of Contents
0. [Our secret key!](#key)
1. [What is Differential Power Analysis? ❓](#dpa)
2. [How does DPA work?](#how)
3. [Differential Power Analysis, from scratch!](#scratch)
    0. [Making things easier...](#easier)
    1. [Building our model / ideal replica](#model)
    2. [Capturing enough traces](#capture)
    3. [DPA's statistical computations - how to find the correct key guess](#stats_comps)
    4. [Is something still unclear?](#unclear)
    5. [Write your DPA implementation!](#implementation)
    6. [What if you are unable to find all the bytes of a key](#unable)
4. [Additional - Plots](#additional)
5. [Conclusions](#conclusions)

#### Before starting, I suggest you running the following snippets of code. They will expand your workspace both horizontally and vertically, helping you to better visualize the plots we are going to create.

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display("text/html", "<style>.container { width:100% !important; }</style>")

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

# 0) Our secret key! <a class="anchor" id="key"></a>
The snippet below defines the secret key we need to find, run it so you'll have it available for the entire notebook. Later, we will compare it with the key retrieved, using DPA, from the `AES-128` implementation running on the target microcontroller.

In [ ]:
secret_key = [0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0X3C]

## 1) What is Differential Power Analysis? ❓ <a class="anchor" id="dpa"></a>
1. [Differential Power Analysis (DPA)](https://en.wikipedia.org/wiki/Power_analysis#Differential_power_analysis)

Differential Power Analysis is the first class of attacks able to retrieve a secret key (or parts of) from a physical implementation of a cipher algorithm. To do so, and contrary to SPA, DPA (and CPA) MUST rely on the **collection of multiple power traces** to succeed. A single power trace will not suffice. This is due to the fact that such attacks rely on statistical computations, which require multiple data instances, with the goal to extract the leakage (i.e. the useful secret data) from each power trace.

> ℹ️ DPA (and CPA, which shares similar principles) are extremely powerful attacks: contrary to SPA, they don't require manual intervention or a visual trace observation: they can be made completely automated and, if the capture environment is optimal, can retrieve parts of a key in a matter of minutes or even less.

Due to the fact that both DPA and CPA leverage statistical computations, it is extremely important to remember the following: **As the number of observations (i.e. captured tracks) increases, so does the amount of leakage we have. Consequently, the greater the leakage, the greater the probability of finding the correct Bytes that make up the secret key. As we said in a previous tutorial, side-channel analysis is impossible to circumvent: given an infinite number of traces collected, Eve will always be able to find the secret key.**

> 📌 The above point is extremely important for every Hardware Hacker: it tells you that, no matter what, **you will always have a non-zero probability of retrieveing a key, and that you can increase your chances of retrieving it just by increasing the number of traces collected.** For instance, if 100 traces give you only 8 Bytes out of a 16 Bytes key, 200 traces, with great chances, can give you (parts of) the remaining ones!

Finally, as a later remark:

> ℹ️ Think about SPA as an attack focusing on **operation-related** observations. The goal of the attacker is to understand what happens during the operating activity of the target microcontroller, i.e. what is the algorithm under execution etc...

> ℹ️ DPA and CPA are different, they achieve **data-related** observations, meaning their goal is to concentrate on the data manipulated in a certain instant of the operating activity of the target device. The goal is to retrieve the key (the data): the operation should be already known by the attacker.

## 2) How does DPA work? <a class="anchor" id="how"></a>
Putting the focus on **data-related** observations may seem quite complex from a physical point of view! 

1. How does an attacker retrieve the value of a key Byte just by looking at the power consumption of a microprocessor? 

First of all, Eve has to know, with great confidence, what is the algorithm she wants to attack. **Both DPA and CPA rely on building a model, or a replica if you prefer, that reflects one specific operation of the main cipher algorithm (let's consider `AES-128`).** Simply put, you cannot attack what you think is `AES-128` when in reality the real algorithm is `RSA`. This is why SPA is so important as a preparatory phase of the attack.

> ℹ️ In the case of `AES-128`, the most common model is the one referring to the `SubBytes` operation. This is the model we will use from now on, in the next image you can see its graphical implementation.

Such a model can be implemented, for instance, with some simple Python code recreating the `aes_internal(key_byte, input_byte)` function in the image below.

<img id="aes_internal" src="./img/04-Differential Power Analysis/replica.png" alt="AES SubBytes Replica" width=600>

Eve knows all the 16 Bytes of plaintext, since she is the one deciding them and feeding them to the microcontroller under attack for each trace captured. She doesn't know the Key, of course, but she knows the entire content of the [`S-Box` matrix](https://en.wikipedia.org/wiki/Rijndael_S-box). Finally, she doesn't know what is the correct value of the result (`0x14` in the above image), but she can somehow try all possible values and relate them to the average power consumption obtained by analyzing all the power traces she collected. But how?

> ℹ️ DPA exploits leakage by correlating it to the value of a variable (`0x14` in the above image) obtained with the ideal replica of the algorithm under attack: that variable is also called "intermediate value" and is the result of an operation that directly involves the key. DPA's rationale bases its assumptions on the existence of the same intermediate value within the real block cipher implementation, which leaves a "footprint" as a specific contribute to the power consumption visible in a power trace.

**I know this sounds complicated**, it is actually a quite complex rationale, and also quite difficult to explain.

---

Basically: you have a "real" system, the one running on the target microcontroller, and a "fake" one: a Python model that replicates parts of the real system's functionalities. 
If you feed them with the same data (assuming your replica is correct), you know **for sure** that both systems will compute the same intermediate results at some point in time. You also know that, in the real system, that intermediate value leaves a "footprint" on the power consumption trace you collected. DPA finds, **with trial and error**, the value of the key that better matches that footprint.

---

We proceed following this simple approach, **based on iterative guessing**, starting by retrieving the first Byte of the key (out of 16):

1. We compute the intermediate value (i.e. the result obtained as output) by feeding the replica with the first plaintext Byte (a known value) and the value `0x00` (our first "guess" for key byte \#0). We save the result obtained;
2. We compute a score, to be assigned to this key guess: this score tells us how well the intermediate value (and hence the key guess) relates to the power traces! Basically, it tells us "how close" the footprint produced by the intermediate matches the footprint visible in the power traces;
2. We repeat the operation now trying `0x01`, then `0x02`, then again `0x03` up until `0xFF`. Once done that, we should have produced 256 intermediates values, collecting 256 key guesses along with their respective scores (256 pairs key_guess - score);
3. We look for the key guess with the highest score: this guess is the one that better relates with the observations obtained from the power traces ➡️ **we found the first byte of the secret key!**
3. We repeat the entire process from point 1), this time considering the second Byte of the key (using the second Byte of the plaintext).
4. We repeat this process for all the 16 Bytes of the key

##### In pseudocode:  <a class="anchor" id="general_pseudocode"></a>

```python
num_key_bytes = 16
keyguess_score_pairs = []
my_key = []

# 100 plaintexts, 16 Bytes each
plaintext_list = [
    [0xB0, 0x3E, 0x0B, 0xAA, 0x05, 0xC1, 0x3A, 0x7E, 0xE0, 0xAB, 0x31, 0xD6, 0x35, 0xF7, 0xC8, 0x8F],
    [..  , ..  , ..  , ..  , ..  , ..  , ..  , ..  , ..  , ..  , ..  , ..  , ..  , ..  , ..  , ..  ],
    ...
    ...
    [0xAE, 0x5F, 0x03, 0x69, 0x13, 0x37, 0x42, 0x0A, 0x13, 0xE4, 0x15, 0x32, 0x4F, 0x82, 0x56, 0xA7]
]

# 100 powertraces, 5000 samples each
powertraces_list = [
    [0.23, -0,3, ..  , ..  , ...   ....   .....   ......   ........   .........   ..........   ....],
    [1,23,  0,4, ..  , ..  , ...   ....   .....   ......   ........   .........   ..........   ....],
    ...
    ...
    [0.02,  0,1, ..  , ..  , ...   ....   .....   ......   ........   .........   ..........   ....]
]

# For all Bytes in a AES-128 key
for i in range(num_key_bytes):
    # For all the values a single Byte can assume
    for key_guess_value in range(0x00, 0xFF):
        # For all the plaintexts arrays sent to the target device
        for plaintext in plaintext_list:
            # Compute the intermediate value using your ideal replica
            intermediate_value = aes_internal(plaintext[i], key_guess_value)
            interm_value_list.append(intermediate_value)
        # end for
        
        # Compute a score to be assigned to the current key guess
        # To do so, we "manipulate" the power traces according 
        # to the values assumed by the intermediates
        # More on that later...
        score = compute_score(interm_value_list, powertraces_list)
        # We save the current key_guess+score pair, to be retrieved later
        keyguess_score_pairs.append( (key_guess_value, score) )
     # end for

    # Now that we tested all possible 256 key guess values, 
    # let's find the one having obtained the highest score.
    # Assuming we collected enough power traces, 
    # the key guess with the highest score is the key byte we were looking for!!
    key_byte = find_best_keyguess(keyguess_score_pairs)
    my_key.append(key_byte)
    
    # You found the *i-th* key byte! Continue looping until you found all of them
# end for

print(f"This is the key I was able to find using DPA: {my_key}")
```

The above approach is what makes DPA/CPA so powerful! If you remember what I told you in the *introduction to SCA* notebook, you may recall the fact that DPA/CPA can disrupt block cipher algorithms like `AES` even if the number of key bits increases! Indeed, the above pseudocode highlights the complexity of the entire algorithm: to find the entire key, a total of $16 \cdot 256$ iterations are needed (16 iterations from the outer loop, the one looping over an entire key, and 256 iterations for the inner loop, the one testing all possible values a key byte can assume). Hence the overall complexity is of $2^4 \cdot 2^8 = 2^{12}$ in the case of a 128 bit key, and a complexity of $2^5 \cdot 2^8 = 2^{13}$ in the case of a 256 bit key (as the outer loop iterates over the $2^5 = 32$ bytes of the key).

1. **How do I know what is the key guess that better matches the footprint left by the real key Byte in the power trace I captured?**

This is the real beauty of the attack! With a high enough number of traces, the statistical computations done in DPA's offline attack can highlight both the presence and the position of the footprint in your power traces!

2. **How do I implement the `compute_score()` function mentioned in the pseudocode above?**

To better understand how, we need to delve into the details of the attack, by simply creating it step by step.

> ⚠️ I assure you that once you understand DPA, CPA is actually way more easy, as it's a simple variation of DPA's main rationale

## 3) Differential Power Analysis, from scratch! <a class="anchor" id="scratch"></a>
The pseudocode above already summarizes how the whole attack is performed. If you understood that, hurray! You are already half way.

We are still lacking, however, the rationale behind the `compute_score()` function, i.e. the statistical computations. **These allows you to sort and find the best key guess among all those tested for the specific key Byte under evaluation.** Basically, they allow you to find the value that best matches the footprint left on the power traces!

In this section, we are going to write a complete DPA attack, from scratch!

### 3.0) Making things easier... <a class="anchor" id="easier"></a>
DPA is quite complex to explain all at once, so let's start with a simpler exercise, and then gradually extend it to cover the totality of the attack.

Hence, instead of attacking an entire 16 Bytes key, let's start by attacking only the first key Byte, Byte \#0

### 3.1) Building our model / ideal replica <a class="anchor" id="model"></a>
As we mentioned early, the attack relies on a golden replica/model to extrapolate all the possible intermediate values that may be computed inside the implementation under attack.
Most of the times, in the case of `AES-128`, the model targets the very first `SubBytes` operation, by replicating the very first `AddRoundKey`/KeyWhitening step and the application of the `S-Box` immediately after. 

> ℹ️ This is done because, with the 128 bit version of `AES`, the first round key corresponds exactly to the master key, i.e. the key upon which all round keys are created. Finding the first round key means finding the master key in a glance, without having to complete the key schedule algorithm in reverse.

Let's build our replica!
We start by defining the content of the SubstitutionBox (`S-Box` matrix), which is known and defined in the `AES` documentation:

In [ ]:
sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

Now, in the following snippet, write the code necessary to define the `aes_internal` function [we saw in one of the previous drawings](#aes_internal). We will use it in the following phases of this tutorial!

In [ ]:
# Define here a function that implements `aes_internal` (2 input values, 1 output value)
# 
# 
#
# 

Let's try your implementation and see if you got it correctly by testing two possible outcomes:

In [ ]:
assert(aes_internal(0x13, 0x37) == 0x36)
assert(aes_internal(0x55, 0xAA) == 0x16)
print("✔️ OK to continue!")

### 3.1) Trace collection <a class="anchor" id="capture"></a>
Let's now collect at least 100 power traces:

In [ ]:
from cyberchallenge_client import ccclient
ccclient.default_capture_config["num_traces"] = 100
connection = ccclient.Utility(str(URL), int(PORT), str(YOUR_TOKEN))
(state, project) = connection.capture_request("firecracker", 60, ccclient.default_capture_config)
print(f"State: {state}")

For each trace captured, we can isolate the `wave`, `textin` (plaintext) and `textout` (ciphertext) objects. For our purposes, only `wave` and `textin` are interesting, as we will attack an `AES-128` encryption. On the contrary, if you want to target a decryption, you'll need to exploit `wave` and `textout`.

In [ ]:
all_textins = [trace.textin for trace in project.traces]
all_waves   = [trace.wave for trace in project.traces]

Let's print 10 of the traces we just collected!

In [ ]:
# Write the code necessary to plot 10 different power traces (waves) on the same graph, try to replicate the image below
#
#
# <YOUR CODE HERE>

<img id="trace_power_differences" src="./img/04-Differential Power Analysis/trace_power_differences.png" alt="Trace Power Differences" width=800>

#### 3.1.1) Understanding the data-related power consumption
Although the previous step was completely unnecessary for the purpose of the attack, having a look to the traces you collected and the small differences among them can help us understand why capturing multiple traces is so important!

> ℹ️ Indeed, you shall notice that, even if all the traces follow the same pattern, some of their samples slightly vary from trace to trace: some rise higher, some have lower values, but all traces follow more or less the same trend. These small variations do not depend on the operation the microcontroller is executing, as it's the same for all the traces collected. Such variations are related only to the data fed as input (the plaintext) and the consequent intermediate values produced during the processor execution!

DPA leverages exactly these small variations, together with a basic but fundamental assumption, that we are going to see in an instant.

### 3.2) DPA's statistical computations - how to find the correct key guess <a class="anchor" id="stats_comps"></a>

#### 3.2.1) Good observations and clever assumptions
We just understood that the power consumption not only varies with the operation performed by the microprocessor under attack, but also with the data handled by it. 

As we mentioned early, DPA bases its observations on the existence of an intermediate value (computed with our ideal replica) that **can be physically correlated to a specific contribute to the power consumption.**

##### Where is this power consumption coming from? What does it depend on? What makes it observable externally through the observation of a power trace?
To reply to all these questions, we must delve into the microprocessor physical implementation, down to the transitor level. Most of the contributions in terms of power consumption are due to ALU computations and data transfers. If we consider the latter, **every single bit transferred on the data bus among the processor core and the register file causes a certain amount of energy to be dissipated, therefore a power consumption, hence a specific contribute inevitably recorded while collecting a power trace.**

> ⚠️ **This entire rationale works as "reductio ad absurdum/argumentum ad absurdum".** We suppose, a priori, that the target bit has some influence on the power traces we collected. The assumption *seems* to make sense, but how do we make sure this happens for real? We force the assumption to be true, *faking 'til we make it*, and see if it helds true or not.

#### 3.2.2) Of prediction functions and target bits

[As we mentioned previously](#general_pseudocode), the complete attack iterates on all the 16 bytes of the key, finding them incrementally. For the sake of simplicity, let's assume we are trying to find the value for key Byte \#0.

The attack proceeds by enumerating all the 256 possible key guesses. The goal is to enumerate them all and find which one of them has the highest chance of being the correct key byte of the secret key. To do so, DPA firstly computes the intermediate value we would have got if that key guess was indeed the correct key byte, and finally correlates it to the power traces collected *. 

But to compute the intermediate value we need two inputs! The key guess, which we have, and the Byte \#0 of the plaintext! This means that we need another loop, a loop that iterates over all the plaintexts collected (remember, every power trace collected comes paired with its plaintext!).

> \* The correlation happens by extrapolating one single bit from the intermediate byte. The value of this bit ('`0'` or `'1'`) is correlated to the consequent presence of a spike in the microcontroller power rail (that you observe when recording a power trace). If this bit is `'1'`, we assume the presence of a spike in the captured power trace, while if the bit is `'0'`, we assume its absence. 

> ℹ️ The bit under observation is generally referred as "target bit". As a tacit convention, generally this bit coincides with **the LSB of the intermediate value**: in reality every bit position can be considered.

This "target bit" policy is what defines the so called **"prediction function"**. 

**This policy is our "argumentum ad absurdum"!**

#### 3.2.3) Of selection functions and partitioning

The following step of the attack involves the so called **"selection function"**. Its goal is to apply the policy dictated by the *prediction function* to the collected power traces.

Assuming the most common policy just mentioned (the LSB one), the selection function evaluates and separates the traces into two distinct sets: the first is associated to the target bit having value `'1'`, while the second relates to the target bit having value `'0'`.

---

Simply put, we extrapolate the target bit from the intermediate value, we check if its value is `'0'` or `'1'` and, consequently, we partition the power trace (the one which is paired to the plaintext we used to get the intermediate) into one of the two sets.

---

At the end of this single evaluation of a key guess (`0x00`, for instance), we should have all the traces partitioned in the two sets.

#### 3.2.4) Highlighting the footprint

Once the partitioning phase has evaluated the entire trace collection, two representative average traces are computed for both of the two sets. Finally, the difference among these traces is determined. What we obtain is a final trace (still with 5000 samples!) with the following properties:

If the choice of which trace is assigned to each set is uncorrelated to the measurements contained in the traces, the difference of the sets’ averages will approach zero as the number of traces increases. Otherwise, if the partitioning into subsets is correlated to the trace measurements, the averages will approach a non-zero value.

The presence of non-zero values in the differential trace indicates that the partitioning phase (and the policy applied) correctly assumed the correlation between the power samples and the value of the observed intermediate bit.

---

**Basically, if we "made the right call" during partitioning (i.e. a good number of traces, for both sets, was indeed assigned to the correct set), then that target bit, thanks to the "difference of the averages" operation, gets "highlighted" as a spike in the final trace!** In other words, all the samples "that have nothing to do" with the target bit (being for their different position in time or their value) gets nullified when doing the average among all the traces of that set. Same goes for the difference among the two average traces.

---

This means that the partitioning phase was correct ➡️ because the target bit demonstrated to be correlated to the power traces ➡️ hence the intermediate value obtained was indeed present in the traces (the target bit is the LSB of the intermediate) ➡️ hence the replica was feeded with the correct data ➡️ and since the plaintext is obviously "correct", i.e. correlated to its power trace ➡️ this means, by exclusion, that the key guess was indeed the right one! **We found the first byte of the key!**

##### Given enough traces, extremely tiny correlations can be isolated no matter how much noise is present in the measurements.

> ℹ️ What happens in the case of a WRONG key guess? Many of the power traces get partitioned in the wrong sets: for instance, a trace having a real contribution may get partitioned in set_0, the one collecting all power traces that SHALL NOT have that contribution! Therefore, at the end of the partitioning phase, the averaging of all traces in a set nullifies the differences among them: if you plot the average trace, you will see some small fluctuations around the `x` axis (meaning the samples all have values around 0) but no big spike.


> ℹ️ What happens in the case of a CORRECT key guess? In such a case, set_1 will indeed contain all those power traces that have a very tiny contribution in their samples due to the target bit presence. On the other hand, set_0 will contain all those power traces that do not show this tiny contribution (because the target bit value is `'0'`). Therefore, at the end of the partitioning phase, the averaging of all traces in a set will highlight the presence of a specific contribution to the power consumption, in both time and value.

Here's a simplified example (8 traces, having only 10 samples each) that better summarizes what happens in case of a CORRECT key guess:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assume you already did the partitioning phase, these are the sets you got:
set_1 = [
    [0.0, 0.2, 2.3, 1.4, -0.2, 1.1, 0.8, 0.1, 0.4, -0.2],
    [0.1, 0.3, 3.2, 1.0, 0.3, 1.0, 0.4, 0.0, 1.0, 0.2],
    [0.0, 0.1, 4.5, 0.2, -0.5, 0.3, -0.2, 0.6, -0.4, 0.1],
    [-0.2, 0.0, 2.9, 0.9, 0.2, -0.2, 0.1, 0.3, 0.5, 1.2],
]
# Compute the average among all traces in set_1
avg_set_1 = np.mean(set_1, axis=0)
# Let's plot all the traces participating in set_1
plt.figure(figsize=[6,4])
plt.grid(visible=True, which='major', axis='both', alpha=0.2)
plt.title("Traces in set_1")
for trace in set_1:
    plt.plot(trace)
    
################################################################
    
set_0 = [
    [0.0, 0.2, -0.2, 0.4, 0.6, 0.5, 0.8, -0.1, 1.0, 0.2],
    [0.1, 0.3, 1.2, 1.0, 0.3, 2.1, -1.4, -0.4, 0.2, 0.9],
    [0.0, 0.1, 0.2, 0.2, -0.5, 1.3, 1.2, -0.6, -0.4, 1.1],
    [-0.2, 0.0, -0.5, 1.3, 1.2, -0.5, 1.1, 0.3, 1.5, 0.8],
]
# Compute the average among all traces in set_0
avg_set_0 = np.mean(set_0, axis=0)
# Let's plot all the traces participating in set_0
plt.figure(figsize=[6,4])
plt.grid(visible=True, which='major', axis='both', alpha=0.2)
plt.title("Traces in set_0")
for trace in set_0:
    plt.plot(trace)

################################################################

# Now compute the difference among the two average traces you just computed
set_difference_trace = np.asarray(avg_set_1 - avg_set_0)

# Plot this difference
plt.figure(figsize=[6,4])
plt.plot(set_difference_trace, label="Difference among sets")
plt.grid(visible=True, which='major', axis='both', alpha=0.2)
plt.legend(title="")

plt.show()

We have a clear spike highlighted at sample \#2! This means that the target bit has a significant correlation with the power traces in that specific point in time.

> ℹ️ Please note, in the above code, the usage of numpy's mean instruction `np.mean(list, axis=0)`. The `axis=0` parameter is really important: it tells numpy to compute the average of each sample **across all traces**, NOT the average of a single trace at a time! For instance, in the above example, the average trace from set_0 will be computed by averaging first sample \#0 (i.e. $\frac{0.0 + 0.1 + 0.0 -0.2}{4}$), across all traces. Once done that, we compute the average of sample \#1, again across all traces, and so on until all the samples in a trace have been evalued.

#### 3.2.5) Assigning a "chance" score to the current key guess

Since we must evaluate all the possible key guesses for that particular key byte (Byte \#0 in our case), it is extremely important to assign a "score" to each key guess. Hence, we must repeat the entire partitioning phase for each key guess, meaning 256 repetitions. For each iteration, we compute the average trace for each of the two sets, we compute their difference and we obtain a final trace, which shall highlight the presence (correlation) of the target bit with the power traces.

##### But how do we assign a score to the key guess given this final trace?
By simply finding the value of the highest peak in the entire trace! This means that, if our power trace has 5000 samples, we need to read each sample starting from `0` up to `4999`, and find the highest one. The highest sample corresponds to the highest peak, which in turn corresponds to the highest point of correlation with the target bit value! Fortunately, the search isn't complicated, and can also be offloaded to numpy.max() function!

> ℹ️ In the above example, the highest peak is, more or less, at 3.0. This means the score assigned to the related key guess will be 3.0!

##### Once assigned a score to all the 256 possible key guess, how do I find the winner?
Simple! The "winning" key guess is the one with the highest score, i.e. the one that got the higher correlation peak!

### 3.3) Is something still unclear? <a class="anchor" id="unclear"></a>
It's fine, read carefully the above explanation, give it another chance, it's hard to understand it the first time.

Also, take a look at the next notebook, [05-Correlation Power Analysis](./05-Correlation%20Power%20Analysis.ipynb): there's a little recap section called **"2.1 Recalling DPA's rationale"** that will summarize all the above. More than often, small recaps allow you to better fix ideas, so leave this notebook for a moment and give it a shot!

### 3.4) Write your DPA implementation! <a class="anchor" id="implementation"></a>
We are at a good point, let's write some pseudocode to summarize this simplified attack (considering only key byte \#0). The real implementation will be up to you!

```python
# Let's concentrate on retrieving the first byte of the key
key_byte_position = 0
# A list of (key_guess, score) tuples, to keep track of the score achieved by each key guess
keyguess_score_pairs = []

# Here we will save the (supposed) value of the key byte we found
my_key_byte = None

# USE THE POWER TRACES AND THEIR RESPECTIVE PLAINTEXTS WE COLLECTED PREVIOUSLY!

For all the possible values a key byte can assume (also called "key guesses")
    # ...
    # Create the two sets as lists
    set_0 = []
    set_1 = []
    # ...
    For all the power traces (waves) we collected previously and their respective plaintexts...
        # Compute the intermediate value using your ideal replica
        # feeding it as input the *first* plaintext byte (because we are working with the *first* key byte) 
        # and the current key guess
        intermediate_value = aes_internal(plaintext[i], key_guess_value)
        # Apply the most common prediction function we mentioned before (LSB) (i.e. find a way to extract the LSB from the intermediate)
        # Now apply the selection function. So, basically...
        if LSB_of_intermediate is '1':
            current_trace goes in set_0
        else:
            current_trace goes in set_1
    # [out of the inner for]
    # ...
    # You just partitioned all the traces collected into two sets, for the current key guess
    # Now compute the average of the two sets (use numpy.mean(set, axis=0), see previous example)
    # In both cases, the result is a trace which is the average of all the traces that participated in that set
    mean_trace_set_0 = average of set_0
    mean_trace_set_1 = average of set_1
    # ...
    # Compute the ABSOLUTE difference among the two average traces 
    # ("absolute" --> meaning that the negative values change sign and become positive)
    abs_difference_trace = numpy.abs(mean_trace_set_0 - mean_trace_set_1)
    # ...
    # Find the value of the maximum spike in the abs_difference_trace
    max_value = numpy.max(abs_difference_trace)
    # ...
    # Assign this value as a score of the current key guess
    keyguess_score_pairs.append( (key_guess, max_value) )
    
# Now find the best key_guess|score pair, i.e. the pair having the highest score
# The key_guess with the highest score is (with good probability) the key byte we were looking for!
key_byte = ...
```

#### Now it's your turn!

Write the code implementing a DPA attack on byte \#0 of the key, following the suggestions of the above pseudocode!
Don't worry about your code complexity, speed etc. The important thing, for your first DPA implementation, is to have a working code! Once you got that, you can always find a way to speed it up later.

#### Implement a complete DPA attack!
Add an outer for loop that iterates over all the 16 Bytes of the key, retrieving them one after the other.

In [ ]:
# ADD YOUR IMPLEMENTATION HERE
#
#
#

### 3.5) What if you are unable to find all the bytes of a key <a class="anchor" id="unable"></a>
Our simple DPA implementation is not flawless: with an extremely good chance, your complete attack will not retrieve completely the key, but parts of it, meaning some **supposedly correct** key bytes found, in reality, do not correspond to the real secret key.

##### How to avoid this?

1. You can increment the number of power traces collected: the higher the number of power observations, the higher the signal-to-noise-ratio and the higher the chances for the attack to assign a higher score to the correct key guess.

2. Instead of picking only a single key guess, the one having the highest absolute score, let's take, for instance, the first 5 highest-ranked key guesses. By doing so, we leave ourselves a few more chances to find the correct key byte. Sure we will now have more work to do in order to validate the entire key, but the process can still be automated and, more important, is still way feasible with modern computers.

3. Change the position of the target bit (e.g., pick the second bit, or the fifth) and/or repeat the attack many times, each time by changing the target bit, and combine the results together.

4. Use a windowing system, i.e. observe the absolute difference trace in small "zoomed-in" portions, called "windows", that focus on a subset of all the samples collected (for instance, a window may go from sample 1000 to sample 1200). If we know where the `AddRoundKey+SubBytes` operations happen, temporally speaking, we can avoid considering other portions of the power traces that have nothing to do with this operation, and that may lead us to consider incorrect key guesses as correct bytes of the secret key.

> ℹ️ You are not required to implement all these approaches, starting from the second on (the first is actually quite easy to achieve). Consider them as a useful optional exercise, you can do them in your free time, nobody's stopping you 😉

> ℹ️ If you decide to implement approach \#3, hold on. You will see that CPA, which does a similar thing, will be faster and achieve better results!

## 4) Additional - Plots <a class="anchor" id="additional"></a>
Try plotting the intermediate traces, try superposing them, learn as much as you can! Add as many code snippets as you want, of course.

Below some examples of plots you can make.

> ⚠️ The graphs below represent a DIFFERENT `AES-128` implementation from the one we are attacking now. These graphs were obtained from a software implementation running on a ChipWhisperer Lite, having an 8-bit AVR as a target microcontroller (different code + different architecture)

In this picture I plotted 10 different power traces, along with the final trace (the absolute difference of the two average traces of the two sets) related to the first **correct** key guess, `0x2B`. You can see how most of the trace values fluctuate around 0, while some peaks indicate the presence of a correlation of the target bit with the power traces. The score assigned to the `0x2bB` key guess comes from the highest peak, the one around sample \#3575.

<img id="[additional]_absDiffTraceComparison1a" src="./img/04-Differential Power Analysis/[additional]_absDiffTraceComparison1a.png" alt="Difference" width=1000>

The same plot as above, only zoomed in to highlight the highest peak, at around sample \#3575. You can also see, in the 10 power traces, the differences among their samples in the exact same temporal position!

<img id="[additional]_absDiffTraceComparison2a" src="./img/04-Differential Power Analysis/[additional]_absDiffTraceComparison2a.png" alt="Difference" width=1000>

## 5) Conclusions <a class="anchor" id="conclusions"></a>

That's it for this tutorial!
 
> I hope you understood how DPA works, and I also hope the explanations were clear, I definitely tried my best but the topic isn't easy!

Next up: Correlation Power Analysis! The next tutorial will be way faster: I will explain you how the attack works, but we will not implement it.

--- 
Do you have suggestions, doubts or simply need to reach me?
* 🐦 Twitter: `[at]mrcuve0`
* 💬 Discord: `Mrcuve0#4179`
* 💻 GitHub:  `Mrcuve0`
* 📬 Email: `mrcuve0 [at] posteo [d.ot] net`
* 💼 LinkedIn:
> Please, send me a message via the previous methods before asking for my LinkedIn

---
Copyright ©️ [CINI - Cybersecurity National Lab](https://cybersecnatlab.it/) - Torino, 2022.

> This material is a derivative work of [NewAE's official Jupyter Notebooks](https://github.com/newaetech/chipwhisperer-jupyter), distributed under the open-source GPL license. You can distribute and modify this material (even for commercial trainings), provided you maintain references to this repository and the original authors, and also offer your derived material under the same conditions.


ChipWhisperer is a trademark of NewAE Technology Inc., claimed in all jurisdictions, and registered in at least the United States of America, European Union, and Peoples Republic of China.

All other product names, logos, and brands are property of their respective owners.

The software is provided 'as is', without warranty of any kind, express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, arising from, out of or in connection with the software or the use or other dealings in the software.